In [1]:
import pathlib

import jax
import jax.numpy as jnp
import numpy as np
from flax import nnx
from jax.sharding import PartitionSpec as P
from rte_dataset.builders import pipeline

from deeprte.configs import default
from deeprte.model import features
from deeprte.model.mapping import inference_subbatch
from deeprte.model.tf import rte_features
from deeprte.train_lib import utils


In [2]:
config = default.Config()

In [3]:
config.load_parameters_path = "/workspaces/deeprte/ckpts/g0.1-test1112/infer/params"

In [4]:
rngs = jax.random.key(0)

In [5]:
# params
data_path = pathlib.Path(
    "/workspaces/deeprte/data/raw_data/test/source-g0.1-qconstant/g0.1-qconstant.npz"
)
data_pipeline = pipeline.DataPipeline(data_path.parent, [data_path.name])
raw_feature_dict = data_pipeline.process()
del data_pipeline

In [6]:
jax.tree.map(lambda x: x.shape if isinstance(x, np.ndarray) else x, raw_feature_dict)

{'functions': {'boundary': (100, 40, 40, 24),
  'boundary_scattering_kernel': (100, 1600, 24, 24),
  'psi_label': (100, 40, 40, 24),
  'scattering_kernel': (100, 40, 40, 24, 24),
  'self_scattering_kernel': (100, 24, 24),
  'sigma': (100, 40, 40, 2),
  'source': (100, 40, 40, 24),
  'source_label': (100, 40, 40, 24)},
 'grid': {'boundary_coords': (40, 40, 24, 4),
  'boundary_weights': (40, 40, 24),
  'phase_coords': (40, 40, 24, 4),
  'position_coords': (40, 40, 2),
  'source_coords': (40, 40, 24, 4),
  'source_weights': (40, 40, 24),
  'velocity_coords': (24, 2),
  'velocity_weights': (24,)},
 'shape': {'num_boundary_coords': 38400,
  'num_examples': 100,
  'num_phase_coords': 38400,
  'num_position_coords': 1600,
  'num_source_coords': 38400,
  'num_velocity_coords': 24}}

In [7]:
i = 0

feature_dict = {
    "functions": jax.tree.map(lambda x: x[i : i + 1], raw_feature_dict["functions"]),
    "grid": raw_feature_dict["grid"],
    "shape": raw_feature_dict["shape"],
}
batch = {}